# TensorFlow 2.0

In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%matplotlib inline

## Hyperparameter Tunning

In [2]:
num_epochs = 20
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [5]:
train_paths = glob('dataset/cifar/train/*.png')[:1000]
test_paths = glob('dataset/cifar/test/*.png')[:1000]

In [6]:
len(train_paths)

1000

In [7]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [8]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [9]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [10]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [11]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [12]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [14]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

# Checkpoint
- tf.keras.callbacks.ModelCheckpoint ( 경로 ~ )

In [15]:
# 새로운 folder명 ( 여기에 checkpoint 들이 저장된다 )
save_path = 'checkpoints'

In [16]:
# 만약 val_accuracy대신 val_loss로 할 경우, mode도 'max'에서 'min'으로 바꿔줘야한다!
checkpoint = tf.keras.callbacks.ModelCheckpoint(save_path, 
                                                monitor='val_accuracy', verbose=1, 
                                                save_best_only=True, mode='max')

## Training

In [18]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    
    validation_data=test_dataset,
    validation_steps=validation_steps,
    
    epochs=num_epochs,
    callbacks=[checkpoint] # checkpoint 여기에 넣기
)

Epoch 1/20
30/31 [============================>.] - ETA: 0s - loss: 2.3000 - accuracy: 0.1165
Epoch 00001: val_accuracy improved from -inf to 0.12298, saving model to checkpoints


W0905 13:28:54.566512 18172 deprecation.py:506] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


31/31 [==============================] - 9s 295ms/step - loss: 2.2990 - accuracy: 0.1147 - val_loss: 2.2811 - val_accuracy: 0.1230
Epoch 2/20
30/31 [============================>.] - ETA: 0s - loss: 2.2341 - accuracy: 0.1645
Epoch 00002: val_accuracy improved from 0.12298 to 0.16230, saving model to checkpoints
31/31 [==============================] - 10s 321ms/step - loss: 2.2341 - accuracy: 0.1674 - val_loss: 2.1942 - val_accuracy: 0.1623
Epoch 3/20
30/31 [============================>.] - ETA: 0s - loss: 2.1896 - accuracy: 0.1774
Epoch 00003: val_accuracy improved from 0.16230 to 0.16835, saving model to checkpoints
31/31 [==============================] - 10s 310ms/step - loss: 2.1884 - accuracy: 0.1798 - val_loss: 2.1853 - val_accuracy: 0.1683
Epoch 4/20
30/31 [============================>.] - ETA: 0s - loss: 2.2036 - accuracy: 0.1870
Epoch 00004: val_accuracy improved from 0.16835 to 0.22681, saving model to checkpoints
31/31 [==============================] - 11s 360ms/step - l

https://www.tensorflow.org/tensorboard/r2/image_summaries#setup